# Obtenemos la primera información de toda la región de la Araucania

In [114]:
import requests
import io
from bs4 import BeautifulSoup
from PyPDF2 import PdfFileReader

## Obtendremos los enlaces y fechas para cada reporte

In [105]:
URL = 'https://seremi9.redsalud.gob.cl/accion-sanitaria-3/registros-covid19/'
page = requests.get(URL) #Obtenemos la respuesta del servidor

In [106]:
soup     = BeautifulSoup(page.content, 'html.parser')  #Transformamos a un objeto soup


In [107]:
job_elems = soup.find_all('ol')                    # Buscamos todos los elementos "ol"
for job_elem in job_elems:                         # Recorremos
    enlaces = job_elem.find_all('a')               # Buscamos los enlaces de ahí

In [108]:
reportes_diarios = []
base_url         = "https://seremi9.redsalud.gob.cl"
for enlace in enlaces:                            # Recorremos los enlaces
    fecha  = enlace.text                          # obtenemos el texto
    enlace = enlace['href']                       # obtenemos el enlace
    url    = base_url + enlace
    dato   = {'fecha': fecha, 'url':url}
    reportes_diarios.append(dato)

## Desde ahora trabajamos con los reportes diarios

In [228]:
#Esta funcion retornara en un arreglo el texto de la pagina 0 del PDF
def get_info(fecha, url):
    print (fecha, url)
    page      = requests.get(url)                           #Obtenemos la respuesta del servidor
    f         = io.BytesIO(page.content)                    #Tenemos el pdf
    reader    = PdfFileReader(f)                            #leemos el pf 
    contenido = reader.getPage(0).extractText().split('\n') #Tenemos el pdf separado por salto de linea
    
    
    #Filtramos e iteramos los datos
    pos_ini   = contenido.index('Angol')
    pos_fin   = contenido.index('Total General')
    
    contenido = contenido[pos_ini:pos_fin]                  # Filtramos con los datos que queremos    
    contenido.remove("Chol")
    contenido.remove("-")
    contenido = ["Chol-Chol" if x=="Chol" else x for x in contenido] #Cambiamos el nombre de cholchol
    
    #Eliminamos los espacios en blanco
    contenido_filtrado = []
    for c in contenido:
        if c != " ":
            contenido_filtrado.append(c)
    
    return contenido_filtrado    

In [237]:
# Retornará si el elemento ingresado es una ciudad
def is_city(elemento):
    bandera  = False
    ciudades = ['Angol', 'Carahue', 'Chol-Chol', 'Collipulli', 'Cunco', 'Curacautín', 'Curarrehue', 'Ercilla', 'Freire', 'Galvarino', 'Gorbea', 'Lautaro', 'Loncoche', 'Lonquimay', 'Los Sauces', 'Lumaco',  'Melipeuco', 'Nueva Imperial', 'Padre Las Casas',  'Perquenco', 'Pitrufquén', 'Pucón', 'Renaico',  'Saavedra', 'Temuco', 'Teodoro Schmidt','Toltén', 'Traiguén',  'Victoria', 'Vilcún', 'Villarrica']
    if elemento in ciudades:
        bandera = True
    return bandera

In [241]:
#Retornará los datos acumulados de los casos por region
def get_data(elemento):
    for e in elemento:
        if is_city(e):
            print (e)

In [242]:
for r in reportes_diarios:
    texto = get_info(r['fecha'], r['url'])
    get_data(texto)
    print ("---------------------------------------------------------------")

01 de Mayo https://seremi9.redsalud.gob.cl/wrdprss_minsal/wp-content/uploads/2020/05/COMUNICADO-DE-PRENSA-01.05.pdf
Angol
Carahue
Chol-Chol
Collipulli
Cunco
Curacautín
Curarrehue
Ercilla
Freire
Galvarino
Gorbea
Lautaro
Loncoche
Lonquimay
Los Sauces
Lumaco
Melipeuco
Nueva Imperial
Padre Las Casas
Perquenco
Pitrufquén
Pucón
Renaico
Saavedra
Temuco
Teodoro Schmidt
Toltén
Traiguén
Victoria
Vilcún
Villarrica
---------------------------------------------------------------
02 de Mayo https://seremi9.redsalud.gob.cl/wrdprss_minsal/wp-content/uploads/2020/05/COMUNICADO-DE-PRENSA-02.05.pdf
Angol
Carahue
Chol-Chol
Collipulli
Cunco
Curacautín
Curarrehue
Ercilla
Freire
Galvarino
Gorbea
Lautaro
Loncoche
Lonquimay
Los Sauces
Lumaco
Melipeuco
Nueva Imperial
Padre Las Casas
Perquenco
Pitrufquén
Pucón
Renaico
Saavedra
Temuco
Teodoro Schmidt
Toltén
Traiguén
Victoria
Vilcún
Villarrica
---------------------------------------------------------------
03 de Mayo https://seremi9.redsalud.gob.cl/wrdprss_minsal/

Angol
Carahue
Chol-Chol
Collipulli
Cunco
Curacautín
Curarrehue
Ercilla
Freire
Galvarino
Gorbea
Lautaro
Loncoche
Lonquimay
Los Sauces
Lumaco
Melipeuco
Nueva Imperial
Padre Las Casas
Perquenco
Pitrufquén
Pucón
Renaico
Saavedra
Temuco
Teodoro Schmidt
Toltén
Traiguén
Victoria
Vilcún
Villarrica
---------------------------------------------------------------
20 de Mayo https://seremi9.redsalud.gob.cl/wrdprss_minsal/wp-content/uploads/2020/05/COMUNICADO-DE-PRENSA-20.05.pdf
Angol
Carahue
Chol-Chol
Collipulli
Cunco
Curacautín
Curarrehue
Ercilla
Freire
Galvarino
Gorbea
Lautaro
Loncoche
Lonquimay
Los Sauces
Lumaco
Melipeuco
Nueva Imperial
Padre Las Casas
Perquenco
Pitrufquén
Pucón
Renaico
Saavedra
Temuco
Teodoro Schmidt
Toltén
Traiguén
Victoria
Vilcún
Villarrica
---------------------------------------------------------------
21 de Mayo https://seremi9.redsalud.gob.cl/wrdprss_minsal/wp-content/uploads/2020/05/COMUNICADO-DE-PRENSA-21.05.pdf
Angol
Carahue
Chol-Chol
Collipulli
Cunco
Curacautín
Curarr